In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import librosa.display
from IPython.display import Audio
import warnings
warnings.filterwarnings('ignore')

import scipy.io.wavfile
import tensorflow


In [2]:
import tensorflow as tensorflow

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization
from tensorflow.keras.layers import Conv1D, MaxPooling1D, AveragePooling1D


In [3]:
get_ipython().system('pip install librosa')
!pip install opendatasets
import opendatasets as od
import os
dai=od.download('https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: kavinvishnun21alr024
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/ejlok1/toronto-emotional-speech-set-tess


100%|██████████| 428M/428M [00:04<00:00, 103MB/s]


'''{"username":"kavinvishnun21alr024","key":"dfc80afdf0f37f079d5b19da8cb51bca"}'''

In [4]:
dataset_dir = '/content/toronto-emotional-speech-set-tess'
paths = []
labels = []
for dirname, _, filenames in os.walk(dataset_dir):
    for filename in filenames:
        paths.append(os.path.join(dirname, filename))
        label = filename.split('_')[-1]
        label = label.split('.')[0]
        labels.append(label.lower())
    if len(paths) == 2800:
        break

print('Dataset is Loaded')

Dataset is Loaded


In [5]:
paths[:5]

['/content/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_angry/OAF_tire_angry.wav',
 '/content/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_angry/OAF_keep_angry.wav',
 '/content/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_angry/OAF_rose_angry.wav',
 '/content/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_angry/OAF_gas_angry.wav',
 '/content/toronto-emotional-speech-set-tess/TESS Toronto emotional speech set data/OAF_angry/OAF_peg_angry.wav']

In [6]:
df = pd.DataFrame()
df['speech'] = paths
df['label'] = labels
df.head()

,speech,label
0,/content/toronto-emotional-speech-set-tess/TES...,angry
1,/content/toronto-emotional-speech-set-tess/TES...,angry
2,/content/toronto-emotional-speech-set-tess/TES...,angry
3,/content/toronto-emotional-speech-set-tess/TES...,angry
4,/content/toronto-emotional-speech-set-tess/TES...,angry


In [7]:
df.loc[df['label'] == 'ps', 'label'] = 'not-stressed'
df.loc[df['label'] == 'happy', 'label'] = 'not-stressed'
df.loc[df['label'] == 'neutral', 'label'] = 'not-stressed'
df.loc[df['label'] == 'sad', 'label'] = 'stressed'
df.loc[df['label'] == 'disgust', 'label'] = 'stressed'
df.loc[df['label'] == 'fear', 'label'] = 'stressed'
df.loc[df['label'] == 'angry', 'label'] = 'stressed'

In [8]:
df['label'].unique()

array(['stressed', 'not-stressed'], dtype=object)

In [9]:
def extract_mfcc(filename):
    y, sr = librosa.load(filename, duration=3, offset=0.5)
    mfcc = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40).T, axis=0)
    return mfcc
extract_mfcc(df['speech'][0])

array([-4.0539273e+02,  8.8814346e+01, -7.8232861e+00, -2.4919470e+01,
       -1.0937450e+01,  1.3049163e+01, -1.9638945e+01, -1.4099646e+01,
       -1.5585397e+01,  1.4451592e+00, -2.1762068e+00,  2.7914851e+00,
       -8.3407342e-01,  2.4630396e+00,  4.0404969e-01,  6.9746560e-01,
       -7.2283134e+00, -1.7304376e+00, -3.8004780e+00, -1.0997511e+01,
       -5.3721437e+00, -1.0431253e+00, -6.7038069e+00,  7.3606215e+00,
        3.4117467e+00,  2.0320349e+01,  2.0861919e+01,  2.7517942e+01,
        1.8740484e+01,  1.3562478e+01, -1.8861243e+00, -4.1360551e-01,
        9.4058132e+00,  7.0190487e+00,  1.0438218e+01, -3.3792951e+00,
        1.6970128e-01,  7.3074617e+00,  3.7013769e+00,  3.6812482e+00],
      dtype=float32)

In [10]:
X_mfcc = df['speech'].apply(lambda x: extract_mfcc(x))
X_mfcc

0       [-405.39273, 88.81435, -7.823286, -24.91947, -...
1       [-486.8117, 60.41997, 4.113057, 8.714228, 11.0...
2       [-430.54114, 80.44542, 6.1626472, -17.457512, ...
3       [-396.6327, 64.15939, -24.143267, -11.737776, ...
4       [-415.30466, 81.50611, -5.9010625, -1.1653156,...
                              ...                        
2795    [-425.66257, 121.42261, 4.703293, 3.8985624, -...
2796    [-405.3745, 105.57376, -7.199868, 11.775073, -...
2797    [-452.0155, 100.76413, 5.724681, 21.546259, -1...
2798    [-378.74265, 98.335754, -15.855036, 24.638155,...
2799    [-398.5753, 116.6968, 8.255272, 5.923474, -24....
Name: speech, Length: 2800, dtype: object

In [11]:
X = [x for x in X_mfcc]
X = np.array(X)
X.shape
X = np.expand_dims(X, -1)
X.shape

(2800, 40, 1)

In [12]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(df[['label']])

In [13]:
y = y.toarray()
print(y)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [14]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [15]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout


model = Sequential([
    Conv1D(64, kernel_size=3, activation='relu', input_shape=(40, 1), padding='same'),
    MaxPooling1D(pool_size=2),
    Conv1D(128, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Conv1D(256, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Conv1D(512, kernel_size=3, activation='relu', padding='same'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(2, activation='softmax')

])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 40, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 20, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 10, 256)             │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 5, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_3 (Conv1D)                    │ (None, 5, 512)              │         393,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 2, 512)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         262,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 820,930 (3.13 MB)

 Trainable params: 820,930 (3.13 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
history = model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=64)

Epoch 1/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - accuracy: 0.5329 - loss: 1.1367 - val_accuracy: 0.7128 - val_loss: 0.5907
Epoch 2/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.6931 - loss: 0.5642 - val_accuracy: 0.8723 - val_loss: 0.3068
Epoch 3/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 96ms/step - accuracy: 0.8962 - loss: 0.2606 - val_accuracy: 0.9468 - val_loss: 0.1518
Epoch 4/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9741 - loss: 0.0857 - val_accuracy: 0.9707 - val_loss: 0.0944
Epoch 5/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.9801 - loss: 0.0499 - val_accuracy: 0.9814 - val_loss: 0.0554
Epoch 6/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.9893 - loss: 0.0366 - val_accuracy: 0.9894 - val_loss: 0.0358
Epoch 7/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 94ms/step - accuracy: 0.9931 - loss: 0.0133 - val_accuracy: 0.9840 - val_loss: 0.0684
Epoch 8/10
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.9916 - loss: 0.0137 - val_accuracy: 0.9894 -

In [18]:
model.save('model.h5')
np.save('train_history.npy', history.history)

In [19]:
from sklearn.metrics import classification_report
preds = model.predict(X_test, batch_size=16, verbose=1)
y_pred = np.argmax(preds, axis=1)  # Convert probabilities to class labels

y_test=np.argmax(y_test,axis=1)

report = classification_report(y_test, y_pred)

print(report)

58/58 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       415
           1       0.99      1.00      1.00       509

    accuracy                           0.99       924
   macro avg       0.99      0.99      0.99       924
weighted avg       0.99      0.99      0.99       924



In [20]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing 

In [21]:
import gradio as gr

def predict_stress(audio_file):

    mfcc = extract_mfcc(audio_file)
    mfcc = np.expand_dims(mfcc, axis=0)
    mfcc = np.expand_dims(mfcc, axis=-1)

    prediction = model.predict(mfcc)
    labels = ['not-stressed', 'stressed']
    emotion = labels[np.argmax(prediction)]
    if emotion=='not-stressed':
      emotion = 'Your not Stressed.Kindly take this in account and be happy as always. '
    else:
      emotion = 'Your Stressed.Kindly take this in account and take care of You.'

    return emotion

In [22]:
iface = gr.Interface(
    fn=predict_stress,
    inputs=[gr.Audio(type="filepath", label="Upload Audio")],
    outputs=[gr.Textbox(label="Generated Text")],
    live=True)


iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7592830465f89c5624.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [24]:
model.save('cnn_model.keras')
